In [1]:
!pip install transformers

     |████████████████████████████████| 1.8MB 19.0MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
     |████████████████████████████████| 3.2MB 57.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ced69aaa90ef6764f68264625cb31923abd996e1e1124734420c645564232de5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!git clone https://github.com/OmarSayedMostafa/Nuanced-Arabic-Dialect-Identification.git

Cloning into 'Nuanced-Arabic-Dialect-Identification'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 220 (delta 121), reused 137 (delta 50), pack-reused 0
Receiving objects: 100% (220/220), 10.38 MiB | 5.17 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [3]:
cd Nuanced-Arabic-Dialect-Identification/

/content/Nuanced-Arabic-Dialect-Identification


In [4]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import re
import random
import string
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch 
import torch.nn as nn
from sklearn.metrics import classification_report, f1_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
# optimizer from hugging face transformers
from transformers import AdamW

In [5]:
pd.set_option('display.max_colwidth', -1)

train_data_path = './cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = './cleaned_data/cleaned_test_data_for_subtask1.csv'

train_dataframe = pd.read_csv(train_data_path)
test_dataframe = pd.read_csv(test_data_path)

In [6]:
train_dataframe.head()

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean_V0,#2_tweet_clean_farasaV0,#2_tweet_clean_V1,#2_tweet_clean_V2,#2_tweet_clean_V3,#classes_id
0,TRAIN_0,حاجة حلوة اكيد,Egypt,حاجة حلوة اكيد,حاج ة حلو ة أكيد,حاجه حلوه اكيد,حاجه حلوه اكيد,حاجه حلوه اكيد,0
1,TRAIN_1,عم بشتغلوا للشعب الاميركي اما نحن يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعدائهم,Iraq,عم بشتغلوا للشعب الاميركي اما نحن يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعدائهم,عم بشتغلوا ل ال شعب ال أميركي اما نحن يكذب وا و يغش وا و يسرق وا و يقتلو شعوب هم و يعمل وا ل صالح اعدائ هم,عم بشتغلوا لشعب الاميركي اما نحن يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعداهم,عم بشتغلوا لشعب الاميركي يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعداهم,عم بشتغلوا لشعب الاميركي يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعداهم,1
2,TRAIN_2,ابشر طال عمرك,Saudi_Arabia,ابشر طال عمرك,ابشر طال عمر ك,ابشر طال عمرك,ابشر طال عمرك,ابشر طال عمرك,2
3,TRAIN_3,منطق 2017: أنا والغريب علي إبن عمي وأنا والغريب وإبن عمي علي أخويا. #قطع_العلاقات_مع_قطر #موريتانيا_مع_قطر,Mauritania,منطق أنا والغريب علي إبن عمي وأنا والغريب وإبن عمي علي أخويا قطع العلاقات مع قطر موريتانيا مع قطر,منطق أنا و ال غريب علي إبن عمي و أنا و ال غريب و إبن عمي علي أخوي ا قطع ال علاق ات مع قطر موريتانيا مع قطر,منطق انا والغريب علي ابن عمي وانا والغريب وابن عمي علي اخويا قطع العلاقات مع قطر موريتانيا مع قطر,منطق والغريب ابن عمي وانا والغريب وابن عمي اخويا قطع العلاقات قطر موريتانيا قطر,منطق والغريب ابن عمي وانا وابن اخويا قطع العلاقات قطر موريتانيا,3
4,TRAIN_4,شهرين وتروح والباقي غير صيف ملينا,Algeria,شهرين وتروح والباقي غير صيف ملينا,شهر ين و تروح و ال باقي غير صيف ملي نا,شهرين وتروح والباقي غير صيف ملينا,شهرين وتروح والباقي صيف ملينا,شهرين وتروح والباقي صيف ملينا,4


In [7]:
classes_names = train_dataframe['#3_country_label'].unique().tolist()

In [8]:
train_x = train_dataframe['#2_tweet_clean_V0'].tolist()
train_y = train_dataframe['#classes_id'].tolist()

test_x = test_dataframe['#2_tweet_clean_V0'].tolist()
test_y = test_dataframe['#classes_id'].tolist()

In [9]:
train_tweets_lengths=[]
for x in train_x:
  if len(x) not in train_tweets_lengths:
    train_tweets_lengths.append(len(x.split(' ')))

test_tweets_lengths=[]
for x in test_x:
  if len(x) not in test_tweets_lengths:
    test_tweets_lengths.append(len(x.split(' ')))

In [10]:
train_tweets_lengths = sorted(train_tweets_lengths)
test_tweets_lengths = sorted(test_tweets_lengths)
train_median_length = np.median(train_tweets_lengths)
test_median_length = np.median(test_tweets_lengths)



print("min train length: ", train_tweets_lengths[0], ",  min test length: ", test_tweets_lengths[0])
print("max train length: ", train_tweets_lengths[-1], ",  max test length: ", test_tweets_lengths[-1])
print("train median length: ", train_median_length, ",  test median length: ",test_median_length)

min train length:  3 ,  min test length:  3
max train length:  63 ,  max test length:  62
train median length:  17.0 ,  test median length:  15.0


In [26]:
tokenizer = AutoTokenizer.from_pretrained("bashar-talafha/multi-dialect-bert-base-arabic")
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_x,
    max_length = int(65),
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    test_x,
    max_length = int(65),
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [37]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_y)

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(test_y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [38]:
# train_dataloader

In [58]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#define a batch size
batch_size = 128
# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)
# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)
# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)
# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)
# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [40]:
bert = AutoModel.from_pretrained("bashar-talafha/multi-dialect-bert-base-arabic")
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [41]:
# batch =  next(iter(train_dataloader)) 
#     # # progress update after every 50 batches.
#     # # push the batch to gpu
#     # # batch = [r.to(device) for r in batch]
# sent_id, mask, labels = batch
# preds = bert(sent_id, mask)

#     # print(len(preds))

#     # break

In [42]:
# len(preds)

In [43]:
# print(preds[0].shape, preds[1].shape)

In [44]:
class BERT_Arch(nn.Module):
    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert 
        # dropout layer
        self.dropout = nn.Dropout(0.4)
        # relu activation function
        self.relu =  nn.ReLU()
        # dense layer 1
        self.fc1 = nn.Linear(768,512)
        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,21)
        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):
        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        # output layer
        x = self.fc2(x)
        # apply softmax activation
        x = self.softmax(x)
        return x

In [45]:
device = 'cuda'
model = BERT_Arch(bert)
model = model.to(device)

In [46]:
# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)
#compute the class weights
y = train_dataframe['#3_country_label']
class_weights = compute_class_weight('balanced', np.unique(y), y)

print(class_weights)

[0.5533307  4.64540421 4.6671117  0.23351927 0.36638368 2.32811633
 2.32811633 1.55087252 0.77724662 4.6671117  1.16814258 0.66628546
 2.33902085 4.6671117  0.46692936 5.80675526 4.6671117  0.77603878
 1.16270303 1.5557039  2.33355585]


In [47]:
# function for evaluating the model
def evaluate():
  print("\nEvaluating...")
  # deactivate dropout layers
  model.eval()
  total_loss, total_accuracy = 0, 0
  # empty list to save the model predictions
  total_preds = []
  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
    # push the batch to gpu
    batch = [t.to(device) for t in batch]
    sent_id, mask, labels = batch
    # deactivate autograd
    with torch.no_grad():
      # model predictions
      preds = model(sent_id, mask)
      # all_predictions.append(preds)
      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
      total_preds.append(preds)
  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds



# function to train the model
def train():
  global model
  model.train()
  total_loss, total_accuracy = 0, 0
  # empty list to save model predictions
  total_preds=[]
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
    # push the batch to gpu
    batch = [r.to(device) for r in batch]
    sent_id, mask, labels = batch
    # clear previously calculated gradients 
    model.zero_grad()        
    # get model predictions for the current batch
    preds = model(sent_id, mask)
    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)
    # add on to the total loss
    total_loss = total_loss + loss.item()
    # backward pass to calculate the gradients
    loss.backward()
    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # update parameters
    optimizer.step()
    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()
    # append the model predictions
    total_preds.append(preds)
  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [62]:
# converting list of class weights to a tensor
# weights= torch.tensor(class_weights,dtype=torch.float)
# push to GPU
# weights = weights.to(device)
# define the loss function
# cross_entropy  = nn.NLLLoss(weight=weights) 
cross_entropy  = nn.NLLLoss() 

# number of training epochs
epochs = 3
# set initial loss to infinite
best_valid_loss = float('inf')
# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]
#for each epoch
for epoch in range(epochs):
  print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
  #train model
  train_loss, _ = train()
  #evaluate model
  valid_loss, all_prediction = evaluate()
  val_prediction = np.argmax(all_prediction, axis=1)
  print(classification_report(val_y, val_prediction, target_names=classes_names))
  macro_f1 = f1_score(val_y, val_prediction, average='macro')
  print("macro_f1:: ", macro_f1)
  #save the best model
  if macro_f1 > best_valid_loss:
      best_valid_loss = macro_f1
      torch.save(model.state_dict(), 'saved_weights_'+str(epoch)+'_f1_'+str(macro_f1)+'_.pt')
  # append training and validation loss
  train_losses.append(train_loss)
  valid_losses.append(valid_loss)
  print(f'\nTraining Loss: {train_loss:.3f}')
  print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 3
  Batch    50  of    164.
  Batch   100  of    164.
  Batch   150  of    164.

Evaluating...
                      precision    recall  f1-score   support

               Egypt       0.59      0.89      0.71      1041
                Iraq       0.35      0.46      0.40       663
        Saudi_Arabia       0.32      0.49      0.39       519
          Mauritania       0.25      0.04      0.07        53
             Algeria       0.37      0.43      0.40       430
               Syria       0.20      0.13      0.16       278
                Oman       0.19      0.20      0.20       355
             Tunisia       0.22      0.12      0.16       172
             Lebanon       0.18      0.08      0.11       157
             Morocco       0.22      0.05      0.08       207
            Djibouti       0.00      0.00      0.00        27
United_Arab_Emirates       0.29      0.24      0.26       157
              Kuwait       0.10      0.04      0.06       105
               Libya    

In [ ]:
ls